# 해당 코드는 employment_full 테이블의 학교, 학과, 취업률을 정규화하는 코드입니다.

- fetch_employments_full.py 실행 후 해당 코드를 실행하면 됩니다.

In [ ]:
!pip install pymysql sqlalchemy

import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# 2. 라이브러리 임포트
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# 3. MySQL 접속 설정
# 아래 정보를 자신의 환경에 맞게 수정해 주세요
user = 'root'
password = 'securityveryimportant'
host = '221.155.195.6'  # 예: 'localhost' 또는 '127.0.0.1'
port = 53306
database = 'main'

In [ ]:
# 테이블 로드
# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# employment_full 불러오기
query_full = "SELECT * FROM employment_full"
df_full = pd.read_sql(query_full, engine)

In [ ]:
# survey_dates 정규화(조사 기준일)
survey_dates_df = df_full[['survey_date']].drop_duplicates().reset_index(drop=True)
survey_dates_df['survey_id'] = survey_dates_df.index + 1

df_full = df_full.merge(survey_dates_df, on='survey_date', how='left')

In [ ]:
# 4. universities 테이블
universities_df = df_full[['school_code', 'school_name', 'edu_level', 'region', 'establishment_type']].drop_duplicates()
universities_df = universities_df.rename(columns={
    'edu_level': 'university_level',
    'establishment_type': 'establishment'
})

In [ ]:
# 5. departments 테이블
departments_df = df_full[['dept_code', 'dept_name', 'school_code', 'major_group', 'mid_major', 'sub_major']].drop_duplicates()
departments_df = departments_df.rename(columns={
    'major_group': 'major_category',
    'mid_major': 'middle_category',
    'sub_major': 'minor_category'
})

In [ ]:
# 6. summary_stats 테이블
summary_stats_df = df_full[['school_code', 'dept_code', 'survey_id',
    'graduates_total', 'graduates_male', 'graduates_female',
    'emp_rate_total', 'emp_rate_male', 'emp_rate_female',
    'advanced_total', 'advanced_male', 'advanced_female',
    'advance_rate_total', 'advance_rate_male', 'advance_rate_female']].drop_duplicates()

In [ ]:
# 7. employment_detail 테이블
detail_columns = [col for col in df_full.columns if col.startswith('취업자_')]
employment_detail_df = df_full[['school_code', 'dept_code', 'survey_id'] + detail_columns].drop_duplicates()

# 컬럼명 변경

In [ ]:
import re

# 취업자 컬럼 목록
korean_columns = [col for col in employment_detail_df.columns if col.startswith('취업자_')]

# 한영 매핑 (필요시 더 추가 가능)
kor2eng = {
    '취업자': 'employed',
    '교외취업자': 'external',
    '교내취업자': 'internal',
    '해외취업자': 'overseas',
    '농림어업종사자': 'agriculture',
    '개인창작활동종사자': 'creator',
    '1인창(사)업자': 'self_employed',
    '프리랜서': 'freelancer',
    '계': 'total',
    '남': 'male',
    '여': 'female',
}

# 변환 함수
def translate_column(kor_col):
    parts = kor_col.split('_')[1:]  # '취업자_' 제거
    translated = [kor2eng.get(p, p) for p in parts]
    return 'employed_' + '_'.join(translated)

# 적용
rename_map = {col: translate_column(col) for col in korean_columns}
employment_detail_df = employment_detail_df.rename(columns=rename_map)

In [ ]:
employment_detail_df.columns.tolist()

['school_code',
 'dept_code',
 'survey_id',
 'employed_external_total',
 'employed_external_male',
 'employed_external_female',
 'employed_internal_total',
 'employed_internal_male',
 'employed_internal_female',
 'employed_overseas_total',
 'employed_overseas_male',
 'employed_overseas_female',
 'employed_agriculture_total',
 'employed_agriculture_male',
 'employed_agriculture_female',
 'employed_creator_total',
 'employed_creator_male',
 'employed_creator_female',
 'employed_self_employed_total',
 'employed_self_employed_male',
 'employed_self_employed_female',
 'employed_freelancer_total',
 'employed_freelancer_male',
 'employed_freelancer_female']

# 확인

In [ ]:
print("universities_df:", universities_df.shape)
print("departments_df:", departments_df.shape)
print("survey_dates_df:", survey_dates_df.shape)
print("summary_stats_df:", summary_stats_df.shape)
print("employment_detail_df:", employment_detail_df.shape)


universities_df: (1619, 5)
departments_df: (23016, 6)
survey_dates_df: (1, 2)
summary_stats_df: (25760, 15)
employment_detail_df: (25543, 24)


In [ ]:
# 1. university FK 무결성 확인
invalid_schools = departments_df.loc[~departments_df['school_code'].isin(universities_df['school_code'])]
print("존재하지 않는 university에 연결된 학과 수:", len(invalid_schools))

# 2. department FK 무결성 확인
invalid_depts = summary_stats_df.loc[~summary_stats_df['dept_code'].isin(departments_df['dept_code'])]
print("존재하지 않는 학과에 연결된 통계 수:", len(invalid_depts))

# 3. survey_id FK 무결성 확인
invalid_surveys = summary_stats_df.loc[~summary_stats_df['survey_id'].isin(survey_dates_df['survey_id'])]
print("존재하지 않는 조사일자에 연결된 통계 수:", len(invalid_surveys))

존재하지 않는 university에 연결된 학과 수: 0
존재하지 않는 학과에 연결된 통계 수: 0
존재하지 않는 조사일자에 연결된 통계 수: 0


In [ ]:
# 중복 확인
print("universities 중복:", universities_df.duplicated().sum())
print("departments 중복:", departments_df.duplicated().sum())
print("survey_dates 중복:", survey_dates_df.duplicated().sum())

universities 중복: 0
departments 중복: 0
survey_dates 중복: 0


# MySQL 테이블 생성 - MySQL에서 할 것
---

🔸 1. univ_survey_dates
```sql
DROP TABLE IF EXISTS univ_survey_dates;

CREATE TABLE univ_survey_dates (
    survey_id INT PRIMARY KEY,
    survey_date DATE
);
```
🔸 2. univ_info
```sql
DROP TABLE IF EXISTS univ_info;

CREATE TABLE univ_info (
    school_code VARCHAR(20) PRIMARY KEY,
    school_name VARCHAR(255),
    university_level VARCHAR(50),
    region VARCHAR(50),
    establishment VARCHAR(50)
);
```
🔸 3. univ_departments
```sql
DROP TABLE IF EXISTS univ_departments;

CREATE TABLE univ_departments (
    dept_code VARCHAR(20) PRIMARY KEY,
    dept_name VARCHAR(255),
    school_code VARCHAR(20),
    major_category VARCHAR(50),
    middle_category VARCHAR(50),
    minor_category VARCHAR(50),
    FOREIGN KEY (school_code) REFERENCES univ_info(school_code)
);
```
🔸 4. univ_summary_stats
```sql
DROP TABLE IF EXISTS univ_summary_stats;

CREATE TABLE univ_summary_stats (
    summary_id INT AUTO_INCREMENT PRIMARY KEY,
    school_code VARCHAR(20),
    dept_code VARCHAR(20),
    survey_id INT,
    graduates_total INT,
    graduates_male INT,
    graduates_female INT,
    emp_rate_total FLOAT,
    emp_rate_male FLOAT,
    emp_rate_female FLOAT,
    advanced_total INT,
    advanced_male INT,
    advanced_female INT,
    advance_rate_total FLOAT,
    advance_rate_male FLOAT,
    advance_rate_female FLOAT,
    FOREIGN KEY (school_code) REFERENCES univ_info(school_code),
    FOREIGN KEY (dept_code) REFERENCES univ_departments(dept_code),
    FOREIGN KEY (survey_id) REFERENCES univ_survey_dates(survey_id)
);
```
🔸 5. univ_employment_detail
```sql
DROP TABLE IF EXISTS univ_employment_detail;

CREATE TABLE univ_employment_detail (
    detail_id INT AUTO_INCREMENT PRIMARY KEY,
    school_code VARCHAR(20),
    dept_code VARCHAR(20),
    survey_id INT,

    employed_external_total INT,
    employed_external_male INT,
    employed_external_female INT,

    employed_internal_total INT,
    employed_internal_male INT,
    employed_internal_female INT,

    employed_overseas_total INT,
    employed_overseas_male INT,
    employed_overseas_female INT,

    employed_agriculture_total INT,
    employed_agriculture_male INT,
    employed_agriculture_female INT,

    employed_creator_total INT,
    employed_creator_male INT,
    employed_creator_female INT,

    employed_self_employed_total INT,
    employed_self_employed_male INT,
    employed_self_employed_female INT,

    employed_freelancer_total INT,
    employed_freelancer_male INT,
    employed_freelancer_female INT,

    FOREIGN KEY (school_code) REFERENCES univ_info(school_code),
    FOREIGN KEY (dept_code) REFERENCES univ_departments(dept_code),
    FOREIGN KEY (survey_id) REFERENCES univ_survey_dates(survey_id)
);
```

#  MySQL 업로드

In [ ]:
# universities_df에서 중복 school_code 제거
universities_df = universities_df.drop_duplicates(subset='school_code')

# 이미 들어간 school_code 확인
with engine.connect() as conn:
    existing_codes = pd.read_sql("SELECT school_code FROM univ_info", conn)
    universities_df = universities_df[~universities_df['school_code'].isin(existing_codes['school_code'])]

# 중복 제거 후 append
universities_df.to_sql('univ_info', con=engine, if_exists='append', index=False)

# 중복 제거
departments_df = departments_df.drop_duplicates(subset='dept_code')

# 이미 DB에 존재하는 dept_code 조회
with engine.connect() as conn:
    existing_dept_codes = pd.read_sql("SELECT dept_code FROM univ_departments", conn)
    departments_df = departments_df[~departments_df['dept_code'].isin(existing_dept_codes['dept_code'])]

# 중복 제거 후 append 시도
departments_df.to_sql('univ_departments', con=engine, if_exists='append', index=False)

# DB에 이미 존재하는 survey_id 불러오기
existing_survey_ids = pd.read_sql("SELECT survey_id FROM univ_survey_dates", con=engine)

# 중복 제거
survey_dates_df_clean = survey_dates_df[~survey_dates_df['survey_id'].isin(existing_survey_ids['survey_id'])]

# 이제 중복 없이 append
survey_dates_df_clean.to_sql('univ_survey_dates', con=engine, if_exists='append', index=False)

summary_stats_df.to_sql('univ_summary_stats', con=engine, if_exists='append', index=False)
employment_detail_df.to_sql('univ_employment_detail', con=engine, if_exists='append', index=False)

25543

In [ ]:
employment_detail_df.columns.tolist()


['school_code',
 'dept_code',
 'survey_id',
 'employed_external_total',
 'employed_external_male',
 'employed_external_female',
 'employed_internal_total',
 'employed_internal_male',
 'employed_internal_female',
 'employed_overseas_total',
 'employed_overseas_male',
 'employed_overseas_female',
 'employed_agriculture_total',
 'employed_agriculture_male',
 'employed_agriculture_female',
 'employed_creator_total',
 'employed_creator_male',
 'employed_creator_female',
 'employed_self_employed_total',
 'employed_self_employed_male',
 'employed_self_employed_female',
 'employed_freelancer_total',
 'employed_freelancer_male',
 'employed_freelancer_female']

# school_code, school_name을 univ_code, univ_name으로 추가 바꿔야함
---
mysql에서 바꿀것.
```
ALTER TABLE univ_info
    CHANGE school_code univ_code VARCHAR(20),
    CHANGE school_name univ_name VARCHAR(255),
    CHANGE university_level univ_level VARCHAR(50);
```